# SCC0230 - Inteligência Artificial (2030)
#### Docente: Solange Oliveira Rezende
#### PAE - Germano Antonio Zani Jorge
#### PAE - Rene Vieira Santin
### **Trabalho 1**
#### Integrantes:
*  Arthur Antonio Rezende Pereira - 15111735
*  Raiad Do Amaral Scaggion - 13727439
*  Rafael Corona - 4769989
*  Vinícius de Moraes - 13749910
*  Vitor Caetano Brustolin - 11795589
*  Yvis Freire Silva Santos - 12608793

### **PARTE 1**

Na USP São Carlos, existem 10 totens espalhados pelo campus 1 com miniaturas dos 8 planetas do sistema solar, o Sol e Plutão. Esses totens representam o sistema solar em escala e mostram aos passantes a real escala do sistema, que geralmente é representada incorretamente em livros didáticos para que seja possível desenhar todos os planetas em uma única página.

Nosso trabalho tem o objetivo de encontrar a menor rota que passa por todos os planetas e retorna ao ponto de início. A solução desse problema pode ser interessante para quem esteja planejanto uma visita educacional pelos totens, na qual todos são visitados e a excursão termina no ponto de início.

O problema foi modelado como um grafo, no qual os vértices são os planetas, as arestas são os caminhos que ligam os planetas e os pesos das arestas são as distâncias de cada caminho. Este problema nada mais é do que uma instância do problema do caixeiro viajante, na qual, a partir de um vértice, é possível atingir outro qualquer diretamente.

Os parâmetros recebidos pelo programa são:
* O número de pontos a serem visitados
* O número de arestas do grafo
* O nome de cada vértice
* Todas as arestas e seus pesos

As variáveis que o programa deseja calcular são:
* Qual a rota mais curta que percorre todos os vértices e retorna ao vértice inicial
* Qual é a distância dessa rota

O vértice inicial do ciclo pode ser qualquer planeta, pois o grafo não é direcionado e a todo momento todos os vértices estão acessíveis.

### Imports
A biblioteca intertools foi utilizada para gerar todos os caminhos possíveis, e a time foi utilizada na aferição de desempenho do código

In [130]:
import itertools
import time
import numpy as np

In [131]:
#classe que representa um grafo valorado não orientado
class WeightedGraph:
    def __init__(self):
        self.graph = {} #listas de adjacencia
        self.N_vertices = 0
        self.N_edges = 0

    #adiciona um vértice ao grafo, caso ele ainda não tenha sido adicionado
    def add_vertex(self, vertex):
        if (vertex not in self.graph):
            self.graph[vertex] = {} #lista de adjacencia do vétice
            self.N_vertices = self.N_vertices+1

    #adiciona uma aresta entre dois vértices do grafo
    def add_edge(self, vertex1, vertex2, weight):
        if (vertex1 in self.graph and vertex2 in self.graph):
            if(weight >= 0):
                self.graph[vertex1][vertex2] = weight
                self.graph[vertex2][vertex1] = weight  # para um grafo não direcionado
            
            #distâncias negativas significam que não há caminho
            else:
                self.graph[vertex1][vertex2] = float('inf')
                self.graph[vertex2][vertex1] = float('inf')
            self.N_edges = self.N_edges+1
    
    #remove uma aresta entre dois vértices do grafo
    def remove_edge(self, vertex1, vertex2):
        if (vertex1 in self.graph and vertex2 in self.graph):
            del self.graph[vertex1][vertex2]
            del self.graph[vertex2][vertex1]# para um grafo não direcionado
            self.N_edges = self.N_edges-1 

    #printa o grafo como uma lista de adjacencias
    def print(self):
        for vertex, neighbors in self.graph.items():
            neighbor_str = ", ".join([f"{neighbor} ({weight})" for neighbor, weight in neighbors.items()])
            print(f"{vertex}: {neighbor_str}")

    #lê um grafo a partir de um arquivo
    def read_from_file(self, file_path):
            with open(file_path, 'r') as file:
                num_vertices = int(file.readline().strip())
                num_edges = int(file.readline().strip())

                for _ in range(num_vertices):
                    vertex = file.readline().strip()
                    self.add_vertex(vertex)

                for _ in range(num_edges):
                    line = file.readline().strip()
                    vertex1, vertex2, weight = line.split()
                    vertex1 = str(vertex1)
                    vertex2 = str(vertex2)
                    weight = float(weight)
                    self.add_edge(vertex1, vertex2, weight)
                #linhas após o termino das arestas são ignoradas

    def convert_to_matrix(self):
        vertices = list(self.graph.keys())
        matrix = []
        for v in vertices:
            matrix.append([])
            for w in range(self.N_vertices):
                matrix[v][w] = self.graph[v][w]
                
    

    #resolve o problema do caixeiro viajante por força bruta
    #retorna o menor caminho e o comprimento desse caminho
    def tsp_bruteforce(self):
        vertices = list(self.graph.keys())

        shortest_path = []
        shortest_distance = float('inf')

        if self.N_vertices < 2:
            return shortest_path, 0  #um único vértice não configura caminho 

        #lista cada permutação possível dos vértices
        for permuted_vertices in itertools.permutations(vertices):
            total_distance = 0
            for i in range(self.N_vertices - 1):
                vertex1 = permuted_vertices[i]
                vertex2 = permuted_vertices[i + 1]
                total_distance += self.graph[vertex1][vertex2]

            # soma as distancias do primeiro ao ultimo vertice
            total_distance += self.graph[permuted_vertices[-1]][permuted_vertices[0]]

            if total_distance < shortest_distance:#escolhe a menor
                shortest_distance = total_distance
                shortest_path = list(permuted_vertices)

        return shortest_path, shortest_distance
    

    def tsp_best_first(self, start_vertex=None):
        if not start_vertex:
            start_vertex = next(iter(self.graph))  # Comecamos do primeiro vertice

        unvisited = set(self.graph.keys()) 
        unvisited.discard(start_vertex) #visitamos o primeiro vertice
        tour = [start_vertex]
        current_vertex = start_vertex

        while unvisited:
            closest_neighbor = None
            min_distance = float('inf')

            for neighbor, weight in self.graph[current_vertex].items():
                if neighbor in unvisited and weight < min_distance:
                    min_distance = weight
                    closest_neighbor = neighbor

            if closest_neighbor is not None:
                tour.append(closest_neighbor)
                unvisited.remove(closest_neighbor)
                current_vertex = closest_neighbor
            else:
                # Caso todos os visinhos ja foram visitados
                # Não ocorre no nosso problema
                current_vertex = tour[0]

        # Soma as distancias
        tour_distance = sum(self.graph[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))
        tour_distance += self.graph[tour[-1]][tour[0]]

        return tour, tour_distance
    
    def tsp_branch_bound(self, distances):
        n = self.N_vertices
        best_path = None
        best_cost = np.inf
            
        # inicializaremos a stack com o no raiz
        stack = [(0, [0], set([0]), 0)]
        while stack:
            node = stack.pop()
            if len(node[1]) == n:
                # se todas os nos foram visitados, atualize o melhor caminho se o caminho novo eh mais curto
                cost = node[3] + distances[int(node[1][-1])][int(0)]
                if cost < best_cost:
                    best_path = node[1]
                    best_cost = cost
            else:
                # gere nos criancas considerando todos os nos nao visitados
                unvisited = set(range(n)) - node[2]
                for i in unvisited:
                    child_path = node[1] + [i]
                    child_cost = node[3] + distances[int(node[1][-1])][int(i)]
                    # podar nos com custo maior do que o nosso melhor custo atual
                    if child_cost < best_cost:
                        stack.append((i, child_path, node[2] | set([i]), child_cost))
        return best_path, best_cost

    # funcao usada para converter a nossa representacao usando listas para uma matrix
    # a fim de facilitar o uso do algoritmo de branch and bound
    def convert_to_matrix(self, matrix):
        vertices = list(self.graph.keys())
        for j in range(self.N_vertices):
            matrix.append([])
            for i in range(self.N_vertices):
                if i != j:
                    vertice1 = vertices[j]
                    vertice2 = vertices[i]
                    matrix[j].append(self.graph[vertice1][vertice2])
                else:
                    matrix[j].append(0)         

O método tsp_bruteforce() gera todas as permutações dos planetas, gerando assim, todos os ciclos que contém todos os planetas. Então realiza uma busca cega pelos caminhos gerados buscando o menor deles. Pela natureza do problema, só é possível saber se uma solução é ótima testando todas as possíveis. Portanto, não é necessário que a busca cega siga algum algoritmo de busca como BFS ou DFS, já que todos os galhos da árvore de busca sempre serão percorridos até o final. Portanto, o código implementado percorre os caminhos na ordem que a função itertools.permutations() os retorna, sem se preocupar com uma ordem específica.

Este algoritmo sempre retorna uma solução ótima, porém é muito lento, tendo uma complexidade algorítmica de O(n!). É o algoritmo com o pior tempo de execução pois nunca realiza podas na árvore.

O método tsp_best_first() Inicia o percurso em um vértice qualquer e a partir daí escolhe o vértice mais próximo e não visitado como o vértice seguinte para continuar o caminho, até que todos os vértices tenham sido visitados.

Este algoritmo tem um ótimo tempo de execução, pois sua complexidade algorítmica é O(n) e é o mais veloz dentre os implementados neste trabalho. Porém não garante a otimalidade da solução e o resultado depende do vértice inicial. É possível que retorne soluções com alto erro relativo em cenários específicos. Mesmo assim, tende a achar soluções razoáveis no geral.

O método tsp_branch_bound() é uma busca informada, e percorre o grafo utilizando uma DFS. Mas ao contrário de uma busca cega, ele armazena a menor solução encontrada até o momento e sempre que um galho atinge um custo maior do que o menor já registrado, este galho é podado. Caso um galho obtenha uma solução menor do que a melhor registrada, a melhor solução é atualizada. Essas podas agilizam muito o tempo de execução e permitem que instâncias maiores do problema possam ser solucionadas.

Este algoritmo sempre retorna uma solução ótima. Apesar de ser melhor do que a solução força bruta, ainda possui complexidade algorítmica de O(n!). Isso ocorre pois nem sempre é possível realizar podas na árvore de buscas. Por exemplo, caso as soluções sejam descobertas em ordem decrescente de custo, nenhuma poda será realizada, e este método será tão lento quanto uma busca cega.

In [132]:
graph = WeightedGraph()

PATH = "data/rotas.txt"
#PATH = "data/euclidian.txt"


graph.read_from_file(PATH)

graph.print()

ME: VE (10.58), TE (19.4), MA (31.02), JU (117.3), SA (239.2), UR (546.68), NE (868.18), PL (1113.0), SO (8.424)
VE: ME (10.58), TE (12.68), MA (25.3), JU (114.6), SA (234.2), UR (473.82), NE (793.17), PL (1003.0), SO (12.68)
TE: ME (19.4), VE (12.68), MA (12.68), JU (102.1), SA (221.6), UR (460.48), NE (780.18), PL (1004.0), SO (24.71)
MA: ME (31.02), VE (25.3), TE (12.68), JU (89.76), SA (89.76), UR (451.74), NE (767.13), PL (1005.0), SO (37.18)
JU: ME (117.3), VE (114.6), TE (102.1), MA (89.76), SA (127.2), UR (368.37), NE (679.21), PL (943.99), SO (125.0)
SA: ME (239.2), VE (234.2), TE (221.6), MA (89.76), JU (127.2), UR (248.25), NE (564.78), PL (827.34), SO (246.0)
UR: ME (546.68), VE (473.82), TE (460.48), MA (451.74), JU (368.37), SA (248.25), NE (289.53), PL (552.08), SO (485.33)
NE: ME (868.18), VE (793.17), TE (780.18), MA (767.13), JU (679.21), SA (564.78), UR (289.53), PL (259.53), SO (816.95)
PL: ME (1113.0), VE (1003.0), TE (1004.0), MA (1005.0), JU (943.99), SA (827.34)

Neste bloco de código o grafo é montado a partir do arquivo de entrada. Pode-se escolher entre o arquivo com as distâncias em linha reta até os planetas ou o arquivo com os caminhos que um pedestre percorreria. Para instâncias muito grandes desse problema, pode ser inviável calcular com precisão os caminhos reais. Para trocar entre os arquivos, basta trocar o comentário do bloco de código acima

### **PARTE 2**

Primeiramente, vamos executar a solução força bruta para obtermos alguns parâmetros iniciais, como a solução ótima e o pior tempo de execução possível 

In [133]:
N_EXECUCOES = 20

start = time.time()
for i in range(N_EXECUCOES):
    shortest_path_bruteforce, shortest_distance_bruteforce = graph.tsp_bruteforce()
end = time.time()

print("Caminho mais curto:", shortest_path_bruteforce)
print("Comprimento total:", "{:.2f}".format(shortest_distance_bruteforce), "metros")
print("Tempo Médio de execução:", "{:.3f}".format((end-start)/N_EXECUCOES),"segundos")

Caminho mais curto: ['MA', 'TE', 'VE', 'SO', 'ME', 'JU', 'NE', 'PL', 'UR', 'SA']
Comprimento total: 1992.59 metros
Tempo Médio de execução: 3.487 segundos


## **Parte 3**

Agora vamos executar os demais algoritmos e aferir suas estatísticas em relação a implementação força bruta. Primeiramente testando com o best first.

In [134]:
start = time.time()
for i in range(N_EXECUCOES):
    shortest_path_bestfirst, shortest_distance_best_first = graph.tsp_best_first()
end = time.time()

print("Caminho mais curto:", shortest_path_bestfirst)
print("Comprimento total:", "{:.2f}".format(shortest_distance_best_first), "metros")
print("Tempo Médio de execução:", "{:.5f}".format((end-start)/N_EXECUCOES),"segundos")
print("Erro relativo da solução: ", "{:.2f}".format((shortest_distance_best_first-shortest_distance_bruteforce)/shortest_distance_bruteforce*100), "%",sep="")

Caminho mais curto: ['ME', 'SO', 'VE', 'TE', 'MA', 'JU', 'SA', 'UR', 'NE', 'PL']
Comprimento total: 2173.73 metros
Tempo Médio de execução: 0.00001 segundos
Erro relativo da solução: 9.09%


Percebemos que o tempo de execução foi muito menor, e que a solução, apesar de não ótima, possui um erro relativo baixo. Note também que esta solução começou em Mercúrio, e caso o ponto de partida fosse outro, a solução provavelmente seria outra

Depois testemos o algoritmo de Branch and Bound.

In [135]:
matrix = []
graph.convert_to_matrix(matrix)
start = time.time()
for i in range(N_EXECUCOES):
    shortest_path_branch_bound, shortest_distance_branch_bound = graph.tsp_branch_bound(matrix)
end = time.time()

#conversão para os nomes dos arquivos
shortest_path_branch_bound[0] = 'ME'
shortest_path_branch_bound[1] = 'VE'
shortest_path_branch_bound[2] = 'TE'
shortest_path_branch_bound[3] = 'MA'
shortest_path_branch_bound[4] = 'JU'
shortest_path_branch_bound[5] = 'SA'
shortest_path_branch_bound[6] = 'UR'
shortest_path_branch_bound[7] = 'NE'
shortest_path_branch_bound[8] = 'PL'
shortest_path_branch_bound[9] = 'SO'

#O caminho começou por outro planeta, porém realiza o mesmo ciclo
print("Caminho mais curto:", shortest_path_branch_bound)
print("Comprimento total:", "{:.2f}".format(shortest_distance_branch_bound), "metros")
print("Tempo Médio de execução:", "{:.5f}".format((end-start)/N_EXECUCOES))
print("Erro relativo da solução: ", "{:.3f}".format((shortest_distance_branch_bound-shortest_distance_bruteforce)/shortest_distance_bruteforce*100), "%", sep="")

Caminho mais curto: ['ME', 'VE', 'TE', 'MA', 'JU', 'SA', 'UR', 'NE', 'PL', 'SO']
Comprimento total: 1992.59 metros
Tempo Médio de execução: 0.09224
Erro relativo da solução: 0.000%


Aqui pode-se perceber que notar que a solução encontrada é ótima e o tempo de execução foi consíderavelmente inferior. Para a instância do problema que nosso grupo se propôs a resolver, o algoritmo branch and bound foi considerado o melhor.